In [0]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns 







In [0]:
# Préprocessing 
data=pd.read_csv(r"/Workspace/Users/thierno.sow@eleves.ec-nantes.fr/customer_shopping_behavior.csv")
data.head
data.describe(include="all")
data.isnull().sum()

data["Review Rating"]=data.groupby(["Category"])["Review Rating"] .transform(lambda x: x.fillna(x.median()) )
data.isnull().sum()


data.columns=data.columns.str.lower()
data.columns=data.columns.str.replace(" ", "_")
data.columns
data=data.rename(columns={"purchase_amount_(usd)": "purchase_amount"})
data.columns


# Creation d'une colonne age_group
def repartition(age):
    if age<25 : 
        return "Young Adult"
    elif age<35 :
        return "Adult"
    elif age<50 :
        return "Middle Aged"
    else:
        return "senior"
data["age_groupe"]=data["age"].apply(repartition)

data.columns


Index(['customer_id', 'age', 'gender', 'item_purchased', 'category',
       'purchase_amount', 'location', 'size', 'color', 'season',
       'review_rating', 'subscription_status', 'shipping_type',
       'discount_applied', 'promo_code_used', 'previous_purchases',
       'payment_method', 'frequency_of_purchases', 'age_groupe'],
      dtype='object')

In [0]:
data.info()

data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3900 entries, 0 to 3899
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customer_id             3900 non-null   int64  
 1   age                     3900 non-null   int64  
 2   gender                  3900 non-null   object 
 3   item_purchased          3900 non-null   object 
 4   category                3900 non-null   object 
 5   purchase_amount         3900 non-null   int64  
 6   location                3900 non-null   object 
 7   size                    3900 non-null   object 
 8   color                   3900 non-null   object 
 9   season                  3900 non-null   object 
 10  review_rating           3900 non-null   float64
 11  subscription_status     3900 non-null   object 
 12  shipping_type           3900 non-null   object 
 13  discount_applied        3900 non-null   object 
 14  promo_code_used         3900 non-null   

,customer_id,age,purchase_amount,review_rating,previous_purchases
count,3900.000000,3900.000000,3900.000000,3900.000000,3900.000000
mean,1950.500000,44.068462,59.764359,3.750051,25.351538
std,1125.977353,15.207589,23.685392,0.713590,14.447125
min,1.000000,18.000000,20.000000,2.500000,1.000000
25%,975.750000,31.000000,39.000000,3.100000,13.000000
50%,1950.500000,44.000000,60.000000,3.800000,25.000000
75%,2925.250000,57.000000,81.000000,4.400000,38.000000
max,3900.000000,70.000000,100.000000,5.000000,50.000000


In [0]:
# purchase_frequency_days
frequency_map={
    "Fortnightly":14 , 
    "Weekly": 7 , 
    "Quarterly":90  , 
    "Bi-weekly":15, 
    "Annualy":365 , 
    "Every 3 months": 90

}
data["purchase_frequency_days"]=data["frequency_of_purchases"].map(frequency_map)
data=data.drop("promo_code_used",axis=1, errors='ignore')
data.columns





Index(['customer_id', 'age', 'gender', 'item_purchased', 'category',
       'purchase_amount', 'location', 'size', 'color', 'season',
       'review_rating', 'subscription_status', 'shipping_type',
       'discount_applied', 'previous_purchases', 'payment_method',
       'frequency_of_purchases', 'age_groupe', 'purchase_frequency_days'],
      dtype='object')

DataFrame enregistré avec succès !


---------------------------------------------------------------------------
UnsupportedOperationException             Traceback (most recent call last)
File <command-6869011401338877>, line 11
      8 dbfs_path = "/FileStore/customers/customer"
     10 # Enregistrer en format Delta (persistant)
---> 11 data_spark.write.format("delta").mode("overwrite").save(dbfs_path)
     13 print(f"DataFrame enregistré dans Databricks à : {dbfs_path}")

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/readwriter.py:703, in DataFrameWriter.save(self, path, format, mode, partitionBy, **options)
    701     self.format(format)
    702 self._write.path = path
--> 703 _, _, ei = self._spark.client.execute_command(
    704     self._write.command(self._spark.client), self._write.observations
    705 )
    706 self._callback(ei)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/client/core.py:1556, in SparkConnectClient.execute_command(self, command, observati